<font color = green >

## Home Task 

</font>


The Task is to extract relevant infromation from the messy medical data using regex.

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

1) Extract the date strings
Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

2) Normalize the extracted dates considering the following rules:  
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

3) Sort records in ascending chronological order accoring.




In [6]:
import os
import numpy as np 
import regex as re
import pandas as pd
cwd= os.getcwd() # current working directory
# path = os.path.join(cwd,'data')
fn=  os.path.join('dates.txt')

with open(fn, 'r') as f:
    content = f.read()
# print (content[:500])
content_copy = content
content_copy = content_copy.split('\n')
# Note: This snippet is extracted just for high level review. You may read the file with the help of pandas or whatever convenient tool 

In [7]:
months_dict = {
    'Jan' : '01',
    'Feb' : '02', 
    'Mar' : '03', 
    'Apr' : '04', 
    'May' : '05', 
    'Jun' : '06', 
    'Jul' : '07', 
    'Aug' : '08', 
    'Sep' : '09', 
    'Oct' : '10', 
    'Nov' : '11', 
    'Dec' : '12',
    '' : ''
}

In [8]:
content_df = pd.DataFrame(content_copy[:500], columns=['res']) 
    
temp = content_df['res'].str.extract(r'(\d{1,2})([-\/])(\d{1,2})([-\/])(\d{2,4})')
content_df['res'] = content_df['res'].str.replace(r'(\d{1,2})([-\/])(\d{1,2})([-\/])(\d{2,4})', '', regex=True)
temp.fillna('', inplace=True)
temp[0] = temp[0].apply(lambda x: '0' + x if len(str(x))== 1 else x)
temp[2] = temp[2].apply(lambda x: '0' + x if len(str(x))== 1 else x)
temp[4] = temp[4].apply(lambda x: '19' + x if len(str(x))== 2 else x)
temp['result'] = temp[4] + '-' + temp[0] + '-' + temp[2]
temp['result'] = temp['result'].apply(lambda x: x if x != '--' else '')
content_df['type_1'] = temp['result']

temp = content_df['res'].str.extract(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)([ .,-]+)(\d{1,2})([ .,-]+)(\d{2,4})')
content_df['res'] = content_df['res'].str.replace(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ .,-]+\d{1,2}[ .,-]+\d{2,4})', '', regex=True)
temp.fillna('', inplace=True)
temp[0] = temp[0].apply(lambda x: x[0:3])
temp[0] = temp[0].map(months_dict, na_action='ignore')
temp['result'] = temp[4] + '-' + temp[0] + '-' + temp[2]
temp['result'] = temp['result'].apply(lambda x: x if x != '--' else '')
content_df['type_2'] = temp['result']

temp = content_df['res'].str.extract(r'(\d{1,2})([ .,\/-]+)((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)([ .,\/-]+)(\d{2,4})')
content_df['res'] = content_df['res'].str.replace(r'(\d{1,2}[ .,\/-]+(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ .,\/-]+\d{2,4})', '', regex=True)
temp.fillna('', inplace=True)
temp[2] = temp[2].apply(lambda x: x[0:3])
temp[2] = temp[2].map(months_dict, na_action='ignore')
temp['result'] = temp[4] + '-' + temp[2] + '-' + temp[0]
temp['result'] = temp['result'].apply(lambda x: x if x != '--' else '')
content_df['type_3'] = temp['result']

temp = content_df['res'].str.extract(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)([ .,/-]+)(\d{2,4})')
content_df['res'] = content_df['res'].str.replace(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ .,/-]+\d{2,4})', '', regex=True)
temp.fillna('', inplace=True)
temp[0] = temp[0].apply(lambda x: x[0:3])
temp[0] = temp[0].map(months_dict, na_action='ignore')
temp['result'] = temp[2] + '-' + temp[0] + '-01'
temp['result'] = temp['result'].apply(lambda x: x if x != '--01' else '')
content_df['type_4'] = temp['result']

temp = content_df['res'].str.extract(r'(\d{1,2})([\/])(\d{4})')
content_df['res'] = content_df['res'].str.replace(r'(\d{1,2})([\/])(\d{4})', '', regex=True)
temp.fillna('', inplace=True)
temp[0] = temp[0].apply(lambda x: '0' + x if len(str(x))== 1 else x)
temp['result'] =  temp[2] + '-' + temp[0] + '-01'
temp['result'] = temp['result'].apply(lambda x: x if x != '--01' else '')
content_df['type_5'] = temp['result']

temp = content_df['res'].str.extract(r'([12]{1}\d{3})')
content_df['res'] = content_df['res'].str.replace(r'(\d{1,2}[\/]\d{4})', '', regex=True)
temp.fillna('', inplace=True)
temp['result'] =  temp[0] + '-01-01'
temp['result'] = temp['result'].apply(lambda x: x if x != '-01-01' else '')
content_df['type_6'] = temp['result']

content_df['res'] = content_df['type_1'] + content_df['type_2'] + content_df['type_3'] + content_df['type_4'] + content_df['type_5'] + content_df['type_6']

content_df = content_df.reset_index()
temp = content_df
content_df = content_df.drop(columns=['type_1', 'type_2', 'type_3', 'type_4', 'type_5', 'type_6', 'index'])
content_df['index'] = temp['index']
content_df = content_df.sort_values(by=['res'])
content_df = content_df.reset_index(drop=True)
content_df
# for i in range(0, 500):
#     print(content_df['index'][i], content_df['result'][i] + '\n')

,res,index
0,1971-04-10,9
1,1971-05-18,84
2,1971-07-08,2
3,1971-07-11,53
4,1971-09-12,28
...,...,...
495,2016-05-01,231
496,2016-05-30,141
497,2016-10-13,186
498,2016-10-19,161


#### <font color = blue >

### Expected Output

</font>

<table align= 'left'>
    <tr><td></td><td> res</td><td>index</td></tr>
    <tr><td>0</td><td> 1971-04-10</td><td>9</td></tr>    
    <tr><td>1</td><td>1971-05-18</td><td>84</td></tr>    
    <tr><td>2</td><td>1971-07-08</td><td>2</td></tr>    
    <tr><td>3</td><td>1971-07-11</td><td>53</td></tr>    
    <tr><td>4</td><td>1971-09-12</td><td>28</td></tr>    
    <tr><td>.</td></tr>    
    <tr><td>.</td></tr>    
    <tr><td>.</td></tr>    
    <tr><td>495 </td><td>2016-05-01</td><td>    427</td></tr>    
    <tr><td>496</td><td> 2016-05-30  </td><td>  141</td></tr>    
    <tr><td>497</td><td> 2016-10-13  </td><td>  186</td></tr>    
    <tr><td>498</td><td> 2016-10-19   </td><td> 161</td></tr>    
    <tr><td>499 </td><td>2016-11-01 </td><td>   413</td></tr>    
</table> 

<font color = green >

## Learn more
</font>

Regular expression operations
<br>
https://docs.python.org/3/library/re.html

Python Regular Expression Tutorial
<br>
https://www.datacamp.com/community/tutorials/python-regular-expression-tutorial

pandas documentation - API Reference - Series 
<br>
https://pandas.pydata.org/pandas-docs/stable/reference/series.html


Regular Expression Analyzer
<br>
http://xenon.stanford.edu/~xusch/regexp/